Make predictions

In [3]:
# # Perform prediction using CPU
# raw_predictions = tft.predict(
#     val_dataloader,
#     mode="raw",
#     return_x=True,  # Include input data for inspection
#     trainer_kwargs={"accelerator": "cpu", "devices": 1}
# )

# # Unpack the Prediction object
# output = raw_predictions[0]  # Network output
# metadata = raw_predictions[1]  # Metadata dictionary

# # Inspect the key elements
# print("Network Output:", output)
# print("Metadata:", metadata)


Plot Predictions

In [7]:
import matplotlib.pyplot as plt

# Extract the first sample's predictions
first_sample_predictions = output.prediction[0]  # Shape: (10, 1)

# Convert predictions to a 1D array for plotting
predictions = first_sample_predictions.squeeze().numpy()  # Shape: (10,)

# Plot the predictions
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(predictions) + 1), predictions, marker='o', linestyle='-', label='Predicted Values')
plt.title('First Sample Predictions Over Forecast Horizon', fontsize=14)
plt.xlabel('Forecast Horizon (Time Steps)', fontsize=12)
plt.ylabel('Predicted Value', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=10)
plt.show()


NameError: name 'output' is not defined

train network


In [8]:
!pip list

Package                    Version      Editable project location
-------------------------- ------------ ---------------------------------------------------------------
absl-py                    2.0.0
aiohttp                    3.9.1
aiosignal                  1.3.1
antlr4-python3-runtime     4.9.3
appdirs                    1.4.4
appnope                    0.1.3
arch                       5.3.1
asttokens                  2.4.1
async-timeout              4.0.3
attrs                      23.2.0
axial-positional-embedding 0.2.1
backcall                   0.2.0
backports.zoneinfo         0.2.1
beautifulsoup4             4.12.3
Brotli                     1.0.9
cachetools                 5.3.2
certifi                    2023.11.17
cftime                     1.6.3
chardet                    5.2.0
charset-normalizer         3.3.2
click                      8.1.7
cmdstanpy                  0.9.68
comm                       0.2.1
contourpy                  1.1.1
convertdate                2.4

In [10]:
import os
import pandas as pd
import numpy as np
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.metrics import SMAPE, MAE
from pytorch_lightning.callbacks import Callback
from torch.utils.data import DataLoader
import torch
from pytorch_lightning import Trainer, LightningModule

# Set device to CUDA if available, otherwise fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# Define a custom callback to log train and validation loss
class LossLoggerCallback(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_losses = []

    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics.get("train_loss", None)
        val_loss = trainer.callback_metrics.get("val_loss", None)
        self.epoch_losses.append((trainer.current_epoch, train_loss, val_loss))
        print(f"Epoch {trainer.current_epoch} -> Train Loss: {train_loss}, Validation Loss: {val_loss}")


# Dataset preparation class
class TimeSeriesDataset:
    def __init__(self, data_folder, context_length, forecast_length):
        self.context_length = context_length
        self.forecast_length = forecast_length
        self.data_files = self.load_csv_files(data_folder)

    def load_csv_files(self, folder):
        all_files = [os.path.join(folder, file) for file in os.listdir(folder) if file.endswith('.csv')]
        datasets = []
        for file in all_files:
            df = pd.read_csv(file, index_col=0)
            df["time_idx"] = range(len(df))
            df["group"] = os.path.basename(file).split('.')[0]  # Use the file name (without extension) as group ID
            datasets.append(df)
        return datasets

    def to_timeseries_dataset(self):
        combined_df = pd.concat(self.data_files, ignore_index=True)
        return combined_df

# LightningModule wrapper for TemporalFusionTransformer
# Wrap the TFT model in the LightningModule
class TFTLightningModule(LightningModule):
    def __init__(self, tft_model):
        super().__init__()
        self.tft_model = tft_model

    def forward(self, x):
        return self.tft_model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = {key: value.to(self.device) for key, value in x.items()}

        # Extract the actual target tensor from the tuple
        y = y[0].to(self.device)

        # Forward pass
        y_hat = self.tft_model(x)
        y_pred = y_hat["prediction"]

        # Reshape predictions if necessary
        if y_pred.ndim > 3:
            y_pred = y_pred.squeeze(-1)

        # Compute loss
        loss = self.tft_model.loss(y_pred, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = {key: value.to(self.device) for key, value in x.items()}

        # Extract the actual target tensor from the tuple
        y = y[0].to(self.device)

        # Forward pass
        y_hat = self.tft_model(x)
        y_pred = y_hat["prediction"]

        # Reshape predictions if necessary
        if y_pred.ndim > 3:
            y_pred = y_pred.squeeze(-1)

        # Compute loss
        loss = self.tft_model.loss(y_pred, y)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.tft_model.hparams.learning_rate)


if __name__ == "__main__":
    # Paths for train and test data
    train_data_folder = "/Users/alecjeffery/Documents/Playgrounds/Python/spacetimeformer_stocks/spacetimeformer/data/train"
    test_data_folder = "/Users/alecjeffery/Documents/Playgrounds/Python/spacetimeformer_stocks/spacetimeformer/data/test"

    context_length = 90
    forecast_length = 10

    # Prepare the train dataset
    train_dataset = TimeSeriesDataset(train_data_folder, context_length, forecast_length)
    train_combined_data = train_dataset.to_timeseries_dataset()
    # Prepare the test dataset
    test_dataset = TimeSeriesDataset(test_data_folder, context_length, forecast_length)
    test_combined_data = test_dataset.to_timeseries_dataset()

    # Ensure 'group' remains a string type
    train_combined_data["group"] = train_combined_data["group"].astype(str)
    test_combined_data["group"] = test_combined_data["group"].astype(str)

    # # Specify additional dataset columns for train
    # train_combined_data["target"] = train_combined_data.iloc[:, 3]  # Assuming the target column is the fourth column
    # # Specify additional dataset columns for test
    # test_combined_data["target"] = test_combined_data.iloc[:, 3]  # Assuming the target column is the fourth column

    train_combined_data = train_combined_data.rename(columns={"Close": "target"})
    test_combined_data = test_combined_data.rename(columns={"Close": "target"})


    # Define the TimeSeriesDataSet for train
    train_ts_dataset = TimeSeriesDataSet(
        train_combined_data,
        time_idx="time_idx",
        target="target",
        group_ids=["group"],
        max_encoder_length=context_length,
        max_prediction_length=forecast_length,
        static_categoricals=["group"],  # 'group' remains a categorical string
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_reals = train_combined_data.drop(columns=['group', 'time_idx']).columns.tolist(),
        # time_varying_unknown_reals=train_combined_data.columns[:].tolist(),#["target"],
        target_normalizer=None,
    )

    # Define the TimeSeriesDataSet for validation
    val_ts_dataset = TimeSeriesDataSet(
        test_combined_data,
        time_idx="time_idx",
        target="target",
        group_ids=["group"],  # 'group' remains a categorical string
        max_encoder_length=context_length,
        max_prediction_length=forecast_length,
        static_categoricals=["group"],
        time_varying_known_reals=["time_idx"],
        # time_varying_unknown_reals=test_combined_data.columns[:].tolist(),
        time_varying_unknown_reals = test_combined_data.drop(columns=['group', 'time_idx']).columns.tolist(),
        target_normalizer=None,
    )

    # Split into dataloaders
    train_dataloader = train_ts_dataset.to_dataloader(
        train=True, batch_size=64, shuffle=True, num_workers=4, persistent_workers=True
    )
    val_dataloader = val_ts_dataset.to_dataloader(
        train=False, batch_size=64, shuffle=False, num_workers=4, persistent_workers=True
    )

    # Define the TemporalFusionTransformer model
    tft = TemporalFusionTransformer.from_dataset(
        train_ts_dataset,
        learning_rate=1e-4,
        lstm_layers=2,
        hidden_size=32,  # 16
        # n_heads=4,
        attention_head_size=16,
        dropout=0.2,
        # max_encoder_length=90,
        # max_prediction_length=10,
        hidden_continuous_size=16,
        output_size=1,  # Single target output
        loss=MAE(),#SMAPE(),
        log_interval=10,
        reduce_on_plateau_patience=4,
    )

    # Wrap the TFT model in the LightningModule
    tft_module = TFTLightningModule(tft)
    tft_module = tft_module.to(device)

    # Print model size
    print(f"Number of parameters in model: {tft.size()}")

    # Use PyTorch Lightning Trainer for training
    trainer = Trainer(
        max_epochs=10,
        gradient_clip_val=0.1,
        enable_checkpointing=True,
        callbacks=[LossLoggerCallback()], 
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1,  # Number of GPUs to use
    )

    # Train the model
    trainer.fit(tft_module, train_dataloader, val_dataloader)

    # Save the trained model
    torch.save(tft_module.state_dict(), "tft_model.pth")

    # # Example inference (requires additional implementation for DataLoader processing)
    # raw_predictions, x = tft.predict(val_dataloader, mode="raw", return_x=True)
    # print("Raw predictions:", raw_predictions)



Using device: cpu


/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/pytorch_forecasting/models/temporal_fusion_transformer/__init__.py:143: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default 

Number of parameters in model: 306319
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

python(6542) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(6601) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(6631) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(6661) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(6715) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(6743) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(6771) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(6773) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 9: 100%|██████████| 3262/3262 [36:58<00:00,  1.47it/s, v_num=4, train_loss_step=0.322, val_loss=0.491, train_loss_epoch=0.311]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 3262/3262 [36:58<00:00,  1.47it/s, v_num=4, train_loss_step=0.322, val_loss=0.491, train_loss_epoch=0.311]


OOS predictions

In [9]:
import os
import pandas as pd
import torch
from pytorch_forecasting.data import TimeSeriesDataSet

# Define the directory holding OOS data
oos_data_folder = "/Users/alecjeffery/Documents/Playgrounds/Python/spacetimeformer_stocks/spacetimeformer/data/oos/"  # Update with the actual path to your OOS directory

# Load all CSV files in the OOS directory
oos_files = [os.path.join(oos_data_folder, f) for f in os.listdir(oos_data_folder) if f.endswith('.csv')]

# Process each file
for file_path in oos_files:
    # Load OOS data
    oos_df = pd.read_csv(file_path, index_col=0)
    oos_df["time_idx"] = range(len(oos_df))  # Ensure `time_idx` is correctly set
    oos_df["group"] = os.path.basename(file_path).split('.')[0]  # Use filename as group ID
    
    # Assuming the target column is the second column in the dataset
    oos_df["target"] = oos_df.iloc[:, 4]
    
    # Prepare TimeSeriesDataSet for OOS data
    oos_ts_dataset = TimeSeriesDataSet(
        oos_df,
        time_idx="time_idx",
        target="target",
        group_ids=["group"],
        max_encoder_length=30,  # Context window (your encoder length)
        max_prediction_length=10,  # Forecast window
        static_categoricals=["group"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_reals=["target"],
        target_normalizer=None,
    )
    
    # Create a DataLoader
    oos_dataloader = oos_ts_dataset.to_dataloader(
        train=False, batch_size=64, shuffle=False, num_workers=4, persistent_workers=True
    )
    
    # Make predictions
    raw_predictions = tft.predict(
        oos_dataloader,
        mode="raw",
        return_x=False,  # We don't need the input data here
        trainer_kwargs={"accelerator": "cpu", "devices": 1},
    )
    
    last_batch_predictions = raw_predictions.prediction[-1]  # The last batch

    # Extract the predictions for the last context window
    final_forecast = last_batch_predictions[-10:]  # All ten forecast periods for the final sample
    print(f"File: {os.path.basename(file_path)}")
    print(f"Final Forecast (Last Context Window -> 10 Periods Ahead): {final_forecast.squeeze().tolist()}")
    print("-" * 40)



/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


File: CSCO.csv
Final Forecast (Last Context Window -> 10 Periods Ahead): [-0.6156679391860962, -0.6164576411247253, -0.6220073103904724, -0.6292522549629211, -0.6378304362297058, -0.6484638452529907, -0.6623948812484741, -0.6810065507888794, -0.7058476805686951, -0.7384976744651794]
----------------------------------------


/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


File: UAL.csv
Final Forecast (Last Context Window -> 10 Periods Ahead): [3.089959144592285, 2.762880563735962, 2.404226303100586, 2.1249845027923584, 1.942463755607605, 1.820357322692871, 1.7271616458892822, 1.6484359502792358, 1.579611897468567, 1.5201222896575928]
----------------------------------------


/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


...

File: CIG.csv
Final Forecast (Last Context Window -> 10 Periods Ahead): [0.34457099437713623, 0.3873754143714905, 0.40325793623924255, 0.4283469319343567, 0.4645920395851135, 0.27904438972473145, -0.5666986703872681, -0.7388328313827515, -0.8941324353218079, -1.0054038763046265]
----------------------------------------


/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


File: CRM.csv
Final Forecast (Last Context Window -> 10 Periods Ahead): [-0.6281646490097046, -0.6661692261695862, -0.693598747253418, -0.7164807915687561, -0.7379088997840881, -0.760023832321167, -0.78492671251297, -0.81472247838974, -0.8514829277992249, -0.8969094157218933]
----------------------------------------


/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


File: PGR.csv
Final Forecast (Last Context Window -> 10 Periods Ahead): [0.7161718010902405, 0.7867752313613892, 0.8994505405426025, 1.0273929834365845, 1.129670262336731, 1.201346516609192, 1.2555010318756104, 1.3026262521743774, 0.6899727582931519, 0.11066900938749313]
----------------------------------------


In [8]:
import os
import pandas as pd
import torch
from pytorch_forecasting.data import TimeSeriesDataSet

# Define the directory holding OOS data
oos_data_folder = "/Users/alecjeffery/Documents/Playgrounds/Python/spacetimeformer_stocks/spacetimeformer/data/oos/"  # Update with the actual path to your OOS directory
# Load all CSV files in the OOS directory
oos_files = [os.path.join(oos_data_folder, f) for f in os.listdir(oos_data_folder) if f.endswith('.csv')]

# Initialize a list to store results
oos_predictions_results = []

# Process each file
for file_path in oos_files:
    # Load OOS data
    oos_df = pd.read_csv(file_path, index_col=0)
    oos_df["time_idx"] = range(len(oos_df))  # Ensure `time_idx` is correctly set
    oos_df["group"] = os.path.basename(file_path).split('.')[0]  # Use filename as group ID
    
    # Assuming the target column is the second column in the dataset
    oos_df["target"] = oos_df.iloc[:, 4]
    
    # Prepare TimeSeriesDataSet for OOS data
    oos_ts_dataset = TimeSeriesDataSet(
        oos_df,
        time_idx="time_idx",
        target="target",
        group_ids=["group"],
        max_encoder_length=context_length,  # Use the same context length as in training
        max_prediction_length=forecast_length,  # Use the same forecast length as in training
        static_categoricals=["group"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_reals=["target"],
        target_normalizer=None,
    )
    
    # Create a DataLoader
    oos_dataloader = oos_ts_dataset.to_dataloader(
        train=False, batch_size=64, shuffle=False, num_workers=4, persistent_workers=True
    )
    
    # Make predictions
    raw_predictions = tft.predict(
        oos_dataloader,
        mode="raw",
        return_x=False,  # We don't need the input data here
        trainer_kwargs={"accelerator": "cpu", "devices": 1},
    )
    
    # Extract the last predictions for this file
    last_predictions = raw_predictions.prediction[:, -1, 0]  # Extract last time step for each sample
    
    # Store results
    oos_predictions_results.append({
        "file": os.path.basename(file_path),
        "last_predictions": last_predictions.tolist()  # Convert tensor to list for better readability
    })

# Print results
for result in oos_predictions_results:
    print(f"File: {result['file']}")
    print(f"Last Predictions: {result['last_predictions']}")
    print("-" * 40)


/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/alecjeffery/anaconda3/envs/spacetimeformer/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


AttributeError: 'list' object has no attribute 'prediction'